In [1]:
## Google Search Example

### 1. **Set up the Google search loader:**

import requests
from bs4 import BeautifulSoup
from langchain.loaders import BaseLoader

ModuleNotFoundError: No module named 'langchain.loaders'